In [1]:
import pandas as pd

In [3]:
import dvc.api

In [ ]:
# python main_inference_multimodal.py \
#         --model=LLM_Inference_Multimodal \
#         --model.llm_type="seqclf" \
#         --model.ckpt_path="models/multitask_multimodal_multilingual/version_13/epoch=2-step=1890.ckpt" \
#         --model.config_path="models/multitask_multimodal_multilingual/version_13/config.yaml" \
#         --model.task="seqclf_singlemodal_wishproducttitle2tax" \
#         --model.output_dir="models/multitask_multimodal_multilingual/version_13" \
#         --model.write_interval="batch" \
#         --data=JSONListData \
#         --data.llm_type="seqclf" \
#         --data.label_map_file="datasets/taxonomy/wish_v1.2.1_newtax_allpaths_withunknown.txt" \
#         --data.label_type="multilabel_taxonomy" \
#         --data.data_source_yaml_path="datasets/multimodal_multitask/liang_product_offshorev2_multilabel_seqclf.yaml" \
#         --data.input_dict="{'template': '{title}', 'task_prefix': 'Classify product title: '}" \
#         --data.output_dict="{'template': '{category_path}'}" \
#         --data.data_source_type="dvc" \
#         --data.model_name="microsoft/Multilingual-MiniLM-L12-H384" \
#         --data.batch_size=50 \
#         --data.max_length=50 \
#         --data.num_workers=0 \
#         --data.overwrite_cache=true \
#         --data.force_download_hfdata=true \
#         --trainer.logger=false \
#         --trainer.enable_checkpointing=false \
#         --trainer.accelerator gpu \
#         --trainer.strategy ddp

In [30]:
df_preds = pd.read_json('/workspaces/multitask-llm-rnd/modelling/models/multitask_multimodal_multilingual/version_13/seqclf-epoch=2-step=1890--liang_product_offshorev2_multilabel_seqclf--test.json', 
                        lines=True)

In [31]:
df_test = pd.read_json(dvc.api.get_url( 
    'datasets/data/liang_product/liang_product_classification_offshorev2only_test.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

In [32]:
df_preds = df_preds.sort_values('batch_indices')

In [33]:
df_tax = pd.read_json(dvc.api.get_url(
    'datasets/data/taxonomy/wish_newtax.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

In [34]:
df_tax_leaf = df_tax[df_tax.is_leaf]

In [35]:
valid_leaf_paths = set(df_tax_leaf['category_path'].str.lower())

In [36]:
df_preds['preds'].apply(lambda x: x[0][0] in valid_leaf_paths).all()

True

In [37]:
df_preds['top_pred'] = df_preds['preds'].apply(lambda x: x[0][0])

In [38]:
df_test.category_path.str.lower().apply(lambda x: x in valid_leaf_paths).all()

True

In [39]:
from eval_from_file import perf_eval_util


In [40]:
df_test['category'] = df_test['category_path'].str.lower().apply(lambda x: x.split(' > '))
df_test['category_pred'] = df_preds['top_pred'].str.lower().apply(lambda x: x.split(' > '))

In [41]:
df_test

,title,description,category_path,category_id,pid,data_type,category,category_pred
0,26 Style Unisex Men Women 2019 Ugly Christmas ...,"Suitable season: Spring,Autumn,winter Fabric n...",Women's Clothing > Sweaters > Sweaters,6037,5d8f06f627ffc134fe3d39da,test_v2_offshore,"[women's clothing, sweaters, sweaters]","[electronic components & supplies, passive com..."
1,Gold Silver Western Texas Longhorn Buffalo Ste...,Condition:100% New Pattern Type:Animal Departm...,Apparel Accessories > Men's Ties & Handkerchiefs,71,58b458051f07f25129092ae9,test_v2_offshore,"[apparel accessories, men's ties & handkerchiefs]","[women's clothing, underwear & sleepwear, wome..."
2,New Quality 1PC 12000uF 63V Amplifier/Audio/Po...,Features: 100% Brand new and high quality. ...,Electronic Components & Supplies > Passive Com...,2122,5d11be0228f36e1d1d3d65be,test_v2_offshore,"[electronic components & supplies, passive com...","[men's clothing, tops & tees, t-shirts]"
3,2019 New Women Shoes Round Toe Flat Bottom Mot...,Welcome to our store! :) Women Shoes Size: U...,Shoes > Women's Shoes > Women's Pumps,4784,5d87170bd54a8c3c7401b476,test_v2_offshore,"[shoes, women's shoes, women's pumps]","[sports, swimming, swimming accessories, myopi..."
4,Whiskey Bottle Beer Mug Super Shape Mylar Foil...,You will receive 2 pcs balloons.The pakage inc...,Home & Garden > Festive & Party Supplies > Eve...,2502,62230f6c1aec6fad16133bb2,test_v2_offshore,"[home & garden, festive & party supplies, even...","[home improvement, lights & lighting, led ligh..."
...,...,...,...,...,...,...,...,...
24805,FISHLUND Braided Fishing Line 100M 300M 500M F...,FISHLUND Fishing Line offers 15 variety of co...,Sports > Fishing > Fishing Lines,5033,62383e08b88420e2246f9ab7,test_v2_offshore,"[sports, fishing, fishing lines]","[hair extensions & wigs, lace wigs, synthetic ..."
24806,"7/8"" 22mm Short 3D CNC Motorcycle Adjustable B...",Features: Upgrade the look with top quality le...,Automobiles & Motorcycles > Motorcycle Accesso...,729,612c8f50e8413e3f7969716d,test_v2_offshore,"[automobiles & motorcycles, motorcycle accesso...","[sports, running, running shorts]"
24807,ZANZEA Cotton Blouse Oversized Women Solid Col...,Package included:1 Blouse Material:100% Cotto...,Women's Clothing > Tops > Blouses & Shirts,6049,620ca17455f3999d7556d235,test_v2_offshore,"[women's clothing, tops, blouses & shirts]","[home & garden, household merchandise, lighter..."
24808,Men's Automatic Quartz Watch Personality Watch...,Movement type: automatic quartz Gender: male T...,Watches > Men's Watches > Quartz Watches,5975,61600766a058c58cd1d5128b,test_v2_offshore,"[watches, men's watches, quartz watches]","[men's clothing, hoodies & sweatshirts]"


In [42]:
perf_eval_util(df_test, 1, 'category_pred')

,id,precision,recall,f1-score,support,level
0,apparel accessories,0.039286,0.041302,0.040268,799.000000,1
1,automobiles & motorcycles,0.083678,0.084995,0.084331,1906.000000,1
2,beauty & health,0.053137,0.053933,0.053532,1335.000000,1
3,cellphones & telecommunications,0.026347,0.026316,0.026332,836.000000,1
4,computer & office,0.014706,0.014388,0.014545,278.000000,1
5,consumer electronics,0.023297,0.022222,0.022747,585.000000,1
6,education & office supplies,0.012012,0.010929,0.011445,366.000000,1
7,electronic components & supplies,0.000000,0.000000,0.000000,28.000000,1
8,entertainment,0.024691,0.023952,0.024316,167.000000,1
9,food,0.000000,0.000000,0.000000,17.000000,1
